In [1]:
import pandas as pd

df = pd.read_csv("Fake_Real_Data.csv")
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [2]:
df.shape

(9900, 2)

In [4]:
df['label'].value_counts()

Fake    5000
Real    4900
Name: label, dtype: int64

In [6]:
df['label_num'] = df['label'].map({'Fake':0,'Real':1})

In [7]:
df.head()

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [8]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [9]:
df['vector'] = df['Text'].apply(lambda x:nlp(x).vector)

In [10]:
df.head()

,Text,label,label_num,vector
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,"[-0.6759837, 1.4263071, -2.318466, -0.451093, ..."
1,U.S. conservative leader optimistic of common ...,Real,1,"[-1.8355803, 1.3101058, -2.4919677, 1.0268308,..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,"[-1.9851209, 0.14389805, -2.4221718, 0.9133005..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,"[-2.7812982, -0.16120885, -1.609772, 1.3624227..."
4,Democrats say Trump agrees to work on immigrat...,Real,1,"[-2.2010763, 0.9961637, -2.4088492, 1.128273, ..."


In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df['vector'],df['label_num'],test_size=0.2,random_state=2022)

In [14]:
x_train

9708    [-2.2631137, 0.30729815, -1.3305324, 0.8236583...
188     [-2.0770946, 0.3782537, -2.353002, -0.25041178...
7932    [-2.4900377, 0.48922613, -2.2653627, 0.8202887...
1686    [-1.2882906, 0.7133769, -1.7222549, -0.3656224...
5625    [-1.0919888, 0.3274793, -1.9463447, 0.59768456...
                              ...                        
6384    [-1.2314047, 1.2643507, -2.6111212, 0.1502925,...
4720    [-1.3940778, 0.99087757, -1.5024365, 0.5952542...
173     [-2.3045952, 0.29060102, -1.654057, 0.602605, ...
1244    [-2.7914732, 0.26514372, -1.7136743, 1.0800198...
4989    [-1.061138, 0.81994104, -2.5229065, -0.3016342...
Name: vector, Length: 7920, dtype: object

In [19]:
import numpy as np
x_train_2d = np.stack(x_train)
x_test_2d =  np.stack(x_test)

In [20]:
x_train_2d

array([[-2.2631137 ,  0.30729815, -1.3305324 , ..., -1.430508  ,
        -1.3995196 ,  0.6761199 ],
       [-2.0770946 ,  0.3782537 , -2.353002  , ..., -0.513723  ,
        -2.7441313 ,  0.7244029 ],
       [-2.4900377 ,  0.48922613, -2.2653627 , ..., -1.849896  ,
        -2.1738544 ,  0.9719141 ],
       ...,
       [-2.3045952 ,  0.29060102, -1.654057  , ..., -1.8792329 ,
        -1.3021306 ,  0.48727357],
       [-2.7914732 ,  0.26514372, -1.7136743 , ..., -1.993619  ,
        -1.2461776 ,  0.9651071 ],
       [-1.061138  ,  0.81994104, -2.5229065 , ...,  0.57455444,
        -2.9443104 ,  1.168517  ]], dtype=float32)

In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(x_train_2d)
scaled_test_embed = scaler.fit_transform(x_test_2d)


clf = MultinomialNB()
clf.fit(scaled_train_embed,y_train)

MultinomialNB()

In [26]:
y_pred = clf.predict(scaled_test_embed)

In [27]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1024
           1       0.94      0.95      0.94       956

    accuracy                           0.94      1980
   macro avg       0.94      0.95      0.94      1980
weighted avg       0.95      0.94      0.94      1980

